In [11]:
%pip install xgboost

     --------------------------------------- 89.1/89.1 MB 12.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [23]:
import mediapipe as mp
import cv2
import numpy as np
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from  xgboost import XGBClassifier
import xgboost
from sklearn.metrics import r2_score
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
columns = ['x','y','visibility','label']
data = {
    'x': [],
    'y': [],
    'visibility': [],
    'label': []
}


In [68]:

# assign directory
directory = r'C:\Users\mosac\Git Repositories\FitnessAI\images'
label = ''
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):

        with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
            frame = cv2.imread(f, cv2.IMREAD_ANYCOLOR)

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
      
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            try:
                landmarks = results.pose_landmarks.landmark
            except:
                pass


            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                            )

        if 'bench' in f:
            label = 'bench'
        elif 'squat' in f:
            label = 'squat'
        else:
            break
        
        
        for i in range(11,len(mp_pose.PoseLandmark)-4):
            data["x"].append(landmarks[i].x)
            data["y"].append(landmarks[i].y)
            data["visibility"].append(landmarks[i].visibility)
            data["label"].append(label)

In [69]:
#Make data array into pandas DataFrame
df = pd.DataFrame(data=data,columns=data.keys())

In [70]:
#look for null values
total_nulls = df.isnull().sum().sum()
print(total_nulls)
#0 nulls found

0


In [71]:
#Encoding categorial features
le = LabelEncoder()
le.fit(df.loc[:,'label'])
df.loc[:,'label'] = le.transform(df.loc[:,'label'])
df.tail()

,x,y,visibility,label
25995,0.478940,0.472293,0.999670,0
25996,0.315266,0.290701,0.985166,0
25997,0.314644,0.316051,0.330695,0
25998,0.163192,0.522137,0.989009,0
25999,0.149374,0.450068,0.633776,0


In [ ]:
#Outlier handling
"""
for i in range(len(df.columns)-5):
  col_name = df.columns[i]
  col = df[col_name]
  Q1 = col.quantile(0.25)
  Q3 = col.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 2 * IQR
  upper_bound = Q3 + 2 * IQR

  # Replace values outside the bounds with the bound values
  col_winsorized = col.clip(lower_bound, upper_bound)

  # Replace the original column with the winsorized data
  df[col_name] = col_winsorized
"""

In [81]:
X = df.drop(['label'],axis=1)
Y = df['label']



X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.0001,random_state=101)

print(f"Train size is: {len(X_train)}\nTest size is: {len(X_test)}")

Train size is: 25997
Test size is: 3


In [83]:
xgb_model = XGBClassifier(learning_rate=0.1, n_estimators= 100,max_depth = 3)

fit = xgb_model.fit(X_train,y_train)

fit_pred = fit.predict(X_train)

xgb_predictions = xgb_model.predict(X_test)

xgb_accuracy = accuracy_score(y_test,xgb_predictions)

#Evaluation - calculating accuracy
xgb_train_acc = accuracy_score(y_train,fit_pred)

#Print accuracy of train and test
print(f"Train Accuracy Is: {round(xgb_train_acc*100,2)}\nTest Accuracy Is: {round(xgb_accuracy*100,2)}")

Train Accuracy Is: 83.42
Test Accuracy Is: 66.67
